In [ ]:
"""
Logistic Regression - PyTorch Implementation
Using autograd for automatic gradient computation.
"""

import numpy as np
import json
import sys
sys.path.append('../..')

# PyTorch
import torch
import torch.nn as nn
import torch.optim as optim

# Self created utilities
from utils.metrics import accuracy, precision, recall, f1_score, auc_score
from utils.performance import track_performance
from utils.visualization import (
    plot_cost_curve,
    plot_confusion_matrix,
    plot_roc_curve,
    plot_feature_importance
)

# Load preprocessed data (already scaled, SMOTE applied, 50/50 balanced)
X_train_np = np.load('../../data/processed/logistic_regression/X_train.npy')
X_test_np = np.load('../../data/processed/logistic_regression/X_test.npy')
y_train_np = np.load('../../data/processed/logistic_regression/y_train.npy')
y_test_np = np.load('../../data/processed/logistic_regression/y_test.npy')

# Load metadata for feature names
with open('../../data/processed/logistic_regression/preprocessing_info.json') as f:
    meta = json.load(f)
feature_names = meta['feature_names']

# NEW SECTION
# Convert to PyTorch tensors
X_train = torch.tensor(X_train_np, dtype=torch.float32)
X_test = torch.tensor(X_test_np, dtype=torch.float32)
y_train = torch.tensor(y_train_np, dtype=torch.float32).reshape(-1, 1)
y_test = torch.tensor(y_test_np, dtype=torch.float32).reshape(-1, 1)

print(f"Training: {X_train.shape[0]:,} samples, {X_train.shape[1]} features")
print(f"Test: {X_test.shape[0]:,} samples")
print(f"Class balance - Train: {y_train.mean().item():.1%} fraud")
print(f"Class balance - Test: {y_test.mean().item():.1%} fraud")